# 5章 誤差逆伝播法

## 単純なレイヤの実装

In [9]:
class MulLayer:
    def __init_(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy

In [10]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 順伝播
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)

220.00000000000003


In [11]:
dprice = 1

# 逆伝播 引数1、mul_tax_layerのx(税抜きのりんごの価格)は200、y(消費税)は1.1
dapple_price, dtax = mul_tax_layer.backward(dprice)
print(dapple_price, dtax)

# 逆伝播 引数1.1 apple_layerのx(りんご)は100、y(個数)は2
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num)


1.1 200
2.2 110.00000000000001


In [12]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        out = x + y
        return out
    
    # 加算レイアの逆伝播は微分を流すだけ
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

## りんご2個とみかん3個の買い物

In [13]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# りんごと個数をかける
apple_price = mul_apple_layer.forward(apple, apple_num)
# オレンジと個数をかける
orange_price = mul_orange_layer.forward(orange, orange_num)
# 税抜き合計金額
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
# 合計金額
price = mul_tax_layer.forward(all_price, tax)

# 逆順に計算していく
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
# 引数の微分がそのままかえる
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
# オレンジの税抜き合計金額の微分からオレンジと個数の微分をもとめる
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
# 上と同じ
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(price)

print(dapple_num, dapple, dorange, dorange_num, dtax)


715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


## 活性化レイヤの実装

In [29]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        print("mask : ", self.mask)
        out = x.copy()
        out[self.mask] = 0 # self.maskのTrueになった番地がoutで0になる。ブールインデックス参照というらしい。
        
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        
        return dx

In [30]:
import numpy as np

r = Relu()
a = r.forward(np.array( [[1.0, -0.5], [-2.0, 3.0]] ))
print(a)

mask :  [[False  True]
 [ True False]]
[[ 1.  0.]
 [ 0.  3.]]


## バッチ版アフィンレイヤ

In [32]:
import numpy as np

X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])

print(X_dot_W)
# [[ 0  0  0]
# [10 10 10]]

print(X_dot_W + B)
# [[ 1  2  3]
# [11 12 13]]

dY = np.array([[1,2,3], [4,5,6]])
print(dY)
# [[1 2 3]
# [4 5 6]]

dB = np.sum(dY, axis=0)
print(dB)
# [5 7 9]
# バイアスの逆伝播は、それぞれのデータの逆伝播の値がバイアスの要素に集約される必要がある、らしい。

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]
[[1 2 3]
 [4 5 6]]
[5 7 9]
